In [2]:
import pandas as pd
import numpy as np

In [3]:
import psycopg2

/Users/michael/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
try:
    conn = psycopg2.connect("dbname=metacritic user=postgres")
except:
    print("cannot connect")

In [5]:
cur = conn.cursor()

In [6]:
import pandas.io.sql as sqlio

In [7]:
sql = "select * from critics;"
critics = sqlio.read_sql_query(sql, conn)

In [8]:
critics.head()

,id,name
0,a189f8ea-e791-4c81-bd22-490cdf4591b1,Logo
1,d5f89497-55c5-4c5c-9272-be319f04b087,The A.V. Club
2,f5ce11dc-eddd-415f-9e12-bb085014c5f6,Exclaim
3,cd9d6f62-d07b-49e8-af6a-0db0773672de,The 405
4,72cdb8d5-cf1e-4ada-8111-f42ddd3e6167,Kerrang!


In [9]:
sql = "select * from albums;"
albums = sqlio.read_sql_query(sql, conn)
#conn = None

In [10]:
albums.head()

,id,name,artist,genres,link,img_link,mc_rating,usr_rating
0,1463a7e4-5bad-482a-9f85-9ac53fce479f,2:54,2:54,"[Pop/Rock, Alternative/Indie Rock, Indie Rock]",/music/254/254,http://static.metacritic.com/images/products/m...,68.0,8.3
1,b3148743-3cc2-41ab-b6c9-67516c0b08eb,The Other I,2:54,"[Pop/Rock, Alternative/Indie Rock, Hard Rock]",/music/the-other-i/254,http://static.metacritic.com/images/products/m...,69.0,NaN
2,0ae002f0-f35c-43d2-ae14-123dcb1b62fa,Plays Music,33.3,"[Indie, Rock, Instrumental]",/music/plays-music/333,http://static.metacritic.com/images/products/m...,51.0,NaN
3,844ffd1e-c437-436f-8547-a57016da09d0,The Hard Way,213,[Rap],/music/the-hard-way/213,http://static.metacritic.com/images/products/m...,56.0,8.0
4,5cd77908-b875-4e9f-aa41-100c08627f08,Don't Tread On Me,311,"[Rock, Alternative]",/music/dont-tread-on-me/311,http://static.metacritic.com/images/products/m...,51.0,6.1


In [56]:
sql = """select ratings.album_name, albums.artist, albums.mc_rating, ratings.critic_id, ratings.album_id from ratings 
JOIN albums ON albums.id = ratings.album_id;"""
ratings = sqlio.read_sql_query(sql, conn)

In [57]:
ratings.head()

,album_name,artist,mc_rating,critic_id,album_id
0,The Monsanto Years,Neil Young,61.0,6e890ea9-d4b9-4976-9d6b-03163632a4be,e26e2612-065a-4437-a4ca-d76e0d093828
1,Venus,Joy Williams,66.0,6e890ea9-d4b9-4976-9d6b-03163632a4be,8b699df4-5f35-44aa-b851-591e0415a8df
2,Teen Men,Teen Men,71.0,6e890ea9-d4b9-4976-9d6b-03163632a4be,c02f8427-58ca-4509-b8c8-26353c04e96d
3,Payola,Desaparecidos,76.0,6e890ea9-d4b9-4976-9d6b-03163632a4be,b1a93cdf-e97c-4e9d-a919-a04db27d00c7
4,Pageant Material,Kacey Musgraves,79.0,6e890ea9-d4b9-4976-9d6b-03163632a4be,089f2f33-c38c-42fe-9c2e-9e4f2c574ee4


In [64]:
rating_totals = pd.DataFrame(ratings.groupby('album_id')['mc_rating'].count())
rating_totals.columns = ['rating_count']
rating_totals['mc_rating'] = ratings.groupby('album_id')['mc_rating'].max()
rating_totals['album_name'] = ratings.groupby('album_id')['album_name'].max()
rating_totals['artist'] = ratings.groupby('album_id')['artist'].max()
rating_totals.head()

,rating_count,mc_rating,album_name,artist
album_id,,,,
000170c8-cc89-418f-85d6-99b2cddbfe2a,6,73.0,Ten Songs from Live at Carnegie Hall,Ryan Adams
00023c63-5cf0-496b-9fed-aedd51e258bd,11,75.0,New Favorite,Alison Krauss & Union Station
0002e56a-0c28-4e7d-a417-70ba6069674b,6,72.0,Thunder Thighs,Kimya Dawson
000cbff7-17f1-4dd8-9cd9-5eb1157fbb95,20,61.0,T.I. Vs T.I.P.,T.I.
0013b996-5b63-4dd3-aeea-d6a05b9f27ed,6,85.0,Traveller,Chris Stapleton


In [14]:
crosstab = pd.pivot_table(data=ratings, values='album_rating', index='critic_id', columns='album_id')

In [15]:
crosstab.head()

album_id,000170c8-cc89-418f-85d6-99b2cddbfe2a,00023c63-5cf0-496b-9fed-aedd51e258bd,0002e56a-0c28-4e7d-a417-70ba6069674b,000cbff7-17f1-4dd8-9cd9-5eb1157fbb95,0013b996-5b63-4dd3-aeea-d6a05b9f27ed,0016bb78-ca68-4364-bc22-34047f61ea04,001f620a-190a-405f-bc11-b1d8d95007eb,00216190-8202-46f3-9952-45b061802075,0023d132-670c-4ac5-8b83-d49b373ec595,0023d215-eae3-4d0e-855c-31562215be26,...,ffcb36a0-3597-4f41-b5e4-2f754b0d32fd,ffd2610f-6e51-418c-9a55-c9df19943795,ffdbe0bc-5319-468a-a971-db16b0ea21d3,ffe6671b-46d2-4d12-aff3-fa88323caef3,ffed69c0-a428-4fb3-bc17-3850dcb70e60,ffee18ab-7a95-4305-984c-4a24ee9497d7,fff1d03a-ff43-48de-993b-e02afb2355cf,fff91846-ec09-4cf2-9446-a58f032d2e44,fffbb171-a1df-418e-83d0-0ecd89a0828e,fffd8d2b-7f94-4d60-a043-ad91f06c68b9
critic_id,,,,,,,,,,,,,,,,,,,,,
06193874-b01f-4bde-be51-7e4ca2b595e0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,...,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,80.0
07c9b103-81e3-425e-bdb0-3ad0b968eff2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0a944835-ed0e-4778-b5b1-1e932970c57d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN
0c63e045-ccc6-453c-9aa4-23b33e6f263d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0eb4e4eb-710d-4335-b214-9be9e8e37383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN


In [16]:
album_name = input("Enter Your Album: ")

Enter Your Album: Funeral


In [17]:
album_name

'Funeral'

In [33]:
pivot_album_id = albums[albums['name']==album_name].id
pivot_album_id = pivot_album_id.max()
#TODO: get album id as string, not object, so it can be passed to next cell programmatically!

In [41]:
album_ratings = crosstab[pivot_album_id]
#albums[albums['id']==pivot_album_id]
#album_ratings[album_ratings>=0]

In [35]:
similar_to_album = crosstab.corrwith(album_ratings)

/Users/michael/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/michael/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


In [42]:
corr_album = pd.DataFrame(similar_to_album, columns=['PearsonR'])
corr_album.dropna(inplace=True)

In [65]:
#TODO: Sort by PearsonR, set minimum number of ratings (join)
corr_summary = corr_album.join(rating_totals)
corr_summary.head()

,PearsonR,rating_count,mc_rating,album_name,artist
album_id,,,,,
000170c8-cc89-418f-85d6-99b2cddbfe2a,-0.007065,6,73.0,Ten Songs from Live at Carnegie Hall,Ryan Adams
00023c63-5cf0-496b-9fed-aedd51e258bd,-0.384349,11,75.0,New Favorite,Alison Krauss & Union Station
000cbff7-17f1-4dd8-9cd9-5eb1157fbb95,-0.431797,20,61.0,T.I. Vs T.I.P.,T.I.
0016bb78-ca68-4364-bc22-34047f61ea04,0.894427,6,88.0,Brutalism,Idles
001f620a-190a-405f-bc11-b1d8d95007eb,0.189476,10,76.0,Forever Today,I'm from Barcelona


In [68]:
corr_summary[corr_summary['rating_count']>=20].sort_values('PearsonR', ascending=False).head(10)

,PearsonR,rating_count,mc_rating,album_name,artist
album_id,,,,,
b5438c07-02b5-41c7-a78d-596b5ba8cc5e,1.000000,33,90.0,Funeral,Arcade Fire
c4972ab7-2c0c-4e5b-ba92-31abadceca77,0.954719,21,81.0,99.90%,Kaytranada
dabc1ead-57d6-47de-a44e-61e7713eac73,0.846261,20,85.0,Elaenia,Floating Points
a3ebe244-a271-44ca-aca7-cbede0c50e6b,0.842979,21,81.0,R.I.P.,Actress
cb7efb96-bb8b-4b0f-b1ea-17a20edf2b88,0.790955,23,76.0,Paper Airplane,Alison Krauss & Union Station
c47da64f-eae2-450b-997f-dd061bb2685d,0.777890,23,84.0,Light Up Gold,Parquet Courts
cab0e275-94ac-48a8-bacd-e3fc686cd408,0.770890,21,76.0,Alopecia,Why?
977b06e3-47bb-48ce-8516-2742697c3914,0.761217,24,79.0,Nepenthe,Julianna Barwick
5f1d4903-21a7-4a19-95a7-07cd52e95f22,0.757002,20,67.0,100 Percent Publishing,Wiley
